In [26]:
import pandas as pd  
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn import metrics
from sklearn.tree import export_graphviz
from sklearn.model_selection import cross_val_score
from pprint import pprint
# from sklearn.decomposition import PCA

pd.options.mode.chained_assignment = None  # default='warn'

### Read in training data

In [2]:
x = pd.read_parquet('training_set_VU_DM.parquet.gzip')

### Assign new weight to booking_bool

In [3]:
x['booking_bool'] = np.where(x['booking_bool'] == 1, 2, 0)

### Create our target class by combining click_bool and booking_bool

In [4]:
x['target'] = x['booking_bool'] + x['click_bool']

### Some feature engineering

In [5]:
# if countries are the same for hotel and user
x['same_location'] = np.where(x['prop_country_id'] == x['visitor_location_country_id'], 1, 0)

# fill nans for prop_location_score2 with prop_location_score1 (since only ~20% missing. note: scaling is different)
multiplier = x['prop_location_score2'].mean() / x['prop_location_score1'].mean()
x['prop_location_score2'].fillna(x['prop_location_score1'] * multiplier, inplace=True)

# fill nans for review score with the mean of all review scores
x['prop_review_score'].fillna(x['prop_review_score'].mean(), inplace=True)

##### THESE WILL BE DELETED LATER, STILL TOO MUCH MISSING DATA AFTERWARDS #####
x['visitor_hist_starrating'].fillna(0, inplace=True)
x['visitor_hist_adr_usd'].fillna(0, inplace=True)
x['has_rate_hist'] = np.where((x['visitor_hist_starrating'] > 0) & (x['visitor_hist_adr_usd'] > 0), 1, 0)

### Fill missing orig_destination_distance data with mean distance for that property id

In [6]:
x["orig_destination_distance"] = x.groupby('prop_id')['orig_destination_distance'].transform(lambda x: x.fillna(x.mean()))
x['orig_destination_distance'].fillna(x['orig_destination_distance'].mean(), inplace=True)

### New features for competitor rates

In [7]:
x_copy = x[['comp1_rate','comp2_rate','comp3_rate','comp4_rate','comp5_rate','comp6_rate','comp7_rate','comp8_rate']].copy()
x['comp_rate_lower'] = (x_copy == -1).T.sum()
x['comp_rate_equal'] = (x_copy == 0).T.sum()
x['comp_rate_higher'] = (x_copy == 1).T.sum()


### New features for competitor availability

In [8]:
x_copy = x[['comp1_inv','comp2_inv','comp3_inv','comp4_inv','comp5_inv','comp6_inv','comp7_inv','comp8_inv']].copy()
x['comp_inv_worse'] = (x_copy == -1).T.sum()
x['comp_inv_same'] = (x_copy == 0).T.sum()
x['comp_inv_better'] = (x_copy == 1).T.sum()

### New features for competitor rate percent difference

In [9]:
x_copy = x[['comp1_rate_percent_diff','comp2_rate_percent_diff','comp3_rate_percent_diff','comp4_rate_percent_diff','comp5_rate_percent_diff','comp6_rate_percent_diff','comp7_rate_percent_diff','comp8_rate_percent_diff']].copy()
x['comp_rate_percent_diff_low'] = x_copy.T.min()
x['comp_rate_percent_diff_high'] = x_copy.T.max()

### Delete the old competitor data

In [10]:
x.drop(['comp1_rate','comp2_rate','comp3_rate','comp4_rate','comp5_rate','comp6_rate','comp7_rate','comp8_rate'], axis=1,inplace=True)
x.drop(['comp1_inv','comp2_inv','comp3_inv','comp4_inv','comp5_inv','comp6_inv','comp7_inv','comp8_inv'], axis=1,inplace=True)
x.drop(['comp1_rate_percent_diff','comp2_rate_percent_diff','comp3_rate_percent_diff','comp4_rate_percent_diff','comp5_rate_percent_diff','comp6_rate_percent_diff','comp7_rate_percent_diff','comp8_rate_percent_diff'], axis=1,inplace=True)

### Combine some competitor features, since some of these columns were quite thin

In [11]:
x['rate_equal_or_better'] = x['comp_rate_equal'] + x['comp_rate_higher']
x['inv_equal_or_better'] = x['comp_inv_same'] + x['comp_inv_better']

### Normalize the columns we will use

In [12]:
# x['prop_starrating'] = MinMaxScaler().fit_transform(np.array(x['prop_starrating'], dtype = float).reshape(-1, 1))
# x['prop_review_score'] = MinMaxScaler().fit_transform(np.array(x['prop_review_score'], dtype = float).reshape(-1, 1))
# x['prop_brand_bool'] = MinMaxScaler().fit_transform(np.array(x['prop_brand_bool'], dtype = float).reshape(-1, 1))
# x['prop_location_score1'] = MinMaxScaler().fit_transform(np.array(x['prop_location_score1'], dtype = float).reshape(-1, 1))
# x['prop_location_score2'] = MinMaxScaler().fit_transform(np.array(x['prop_location_score2'], dtype = float).reshape(-1, 1))
# x['prop_log_historical_price'] = MinMaxScaler().fit_transform(np.array(x['prop_log_historical_price'], dtype = float).reshape(-1, 1))
# x['price_usd'] = x.groupby('prop_country_id')['price_usd'].transform(lambda x: (x - min(x)) / (max(x) - min(x)))
# x['promotion_flag'] = MinMaxScaler().fit_transform(np.array(x['promotion_flag'], dtype = float).reshape(-1, 1))
# x['orig_destination_distance'] = MinMaxScaler().fit_transform(np.array(x['orig_destination_distance'], dtype = float).reshape(-1, 1))
# x['random_bool'] = MinMaxScaler().fit_transform(np.array(x['random_bool'], dtype = float).reshape(-1, 1))
# x['same_location'] = MinMaxScaler().fit_transform(np.array(x['same_location'], dtype = float).reshape(-1, 1))
# x['rate_equal_or_better'] = MinMaxScaler().fit_transform(np.array(x['rate_equal_or_better'], dtype = float).reshape(-1, 1))
# x['inv_equal_or_better'] = MinMaxScaler().fit_transform(np.array(x['inv_equal_or_better'], dtype = float).reshape(-1, 1))

### mean, median and std for prop\_starrating, prop\_review\_score, prop\_location\_score1, and prop\_location\_score2 for each hotel (grouped by "prop\_id"

In [13]:
features = ["prop_starrating", "prop_review_score", "prop_location_score1", "prop_location_score2"]

for feat in features: 
    mean = x.groupby("prop_id")[feat].mean()
    median = x.groupby("prop_id")[feat].median()
    std = x.groupby("prop_id")[feat].std(ddof=0)
    
#     print (std.min())
#     print (std.mean())
#     print (std.max())

    x[feat+"_mean"] = mean[x.prop_id].values
    x[feat+"_median"] = median[x.prop_id].values
    x[feat+"_std"] = std[x.prop_id].values

### Delete columns we do not need

In [14]:
del x['date_time']
# del x['site_id']
del x['booking_bool']
del x['click_bool']
del x['gross_bookings_usd']
# del x['visitor_location_country_id']
# del x['prop_country_id']
del x['visitor_hist_starrating']
del x['visitor_hist_adr_usd']
del x['srch_query_affinity_score']
del x['position']
#del x['prop_id']
del x['srch_id']
# del x['srch_destination_id']
# del x['srch_booking_window']
# del x['srch_adults_count']
# del x['srch_children_count']
# del x['srch_room_count']
# del x['srch_saturday_night_bool']
# del x['srch_length_of_stay']
del x['comp_rate_percent_diff_low']
del x['comp_rate_percent_diff_high']
del x['comp_rate_equal']
del x['comp_rate_higher']
del x['comp_inv_same']
del x['comp_inv_better']

### Delete the competitor low/high rate percent differences for now, since ~68% of entries have no values at all. Also delete users that have booked & rated in the past, since ~95% of this is still empty

In [15]:
# del x['comp_rate_percent_diff_low']
# del x['comp_rate_percent_diff_high']
#del x['has_rate_hist']

### Try deleting some columns to see what happens

In [16]:
#del x['prop_log_historical_price']
#del x['random_bool']

### Check final column list

In [17]:
x.columns

Index([                    u'site_id', u'visitor_location_country_id',
                   u'prop_country_id',                     u'prop_id',
                   u'prop_starrating',           u'prop_review_score',
                   u'prop_brand_bool',        u'prop_location_score1',
              u'prop_location_score2',   u'prop_log_historical_price',
                         u'price_usd',              u'promotion_flag',
               u'srch_destination_id',         u'srch_length_of_stay',
               u'srch_booking_window',           u'srch_adults_count',
               u'srch_children_count',             u'srch_room_count',
          u'srch_saturday_night_bool',   u'orig_destination_distance',
                       u'random_bool',                      u'target',
                     u'same_location',               u'has_rate_hist',
                   u'comp_rate_lower',              u'comp_inv_worse',
              u'rate_equal_or_better',         u'inv_equal_or_better',
      

### PCA

In [18]:
# pca = PCA(n_components='mle', svd_solver='full')
# pca.fit(x)
# pca.explained_variance_ratio_
# pca.singular_values_

In [19]:
# x2 = pd.DataFrame(pca.transform(x))

In [20]:
#x.columns = ['site_id','prop_country_id','prop_starrating','prop_brand_bool','prop_location_score2','price_usd','srch_destination_id','srch_booking_window','srch_children_count','srch_saturday_night_bool','random_bool','same_location','comp_rate_lower','rate_equal_or_better','prop_starrating_mean','prop_starrating_std','prop_review_score_median','prop_location_score1_mean','prop_location_score1_std','prop_location_score2_median','visitor_location_country_id','prop_id','prop_review_score','prop_location_score1','prop_log_historical_price','promotion_flag','srch_length_of_stay','srch_adults_count','srch_room_count','orig_destination_distance','target','has_rate_hist','comp_inv_worse','inv_equal_or_better','prop_starrating_median','prop_review_score_mean','prop_review_score_std','prop_location_score1_median','prop_location_score2_mean','prop_location_score2_std']
# x['target'] = target

In [21]:
x.head

<bound method DataFrame.head of          site_id  visitor_location_country_id  prop_country_id  prop_id  \
0             12                          187              219      893   
1             12                          187              219    10404   
2             12                          187              219    21315   
3             12                          187              219    27348   
4             12                          187              219    29604   
5             12                          187              219    30184   
6             12                          187              219    44147   
7             12                          187              219    50984   
8             12                          187              219    53341   
9             12                          187              219    56880   
10            12                          187              219    59267   
11            12                          187              219    59

In [22]:
x.columns

Index([                    u'site_id', u'visitor_location_country_id',
                   u'prop_country_id',                     u'prop_id',
                   u'prop_starrating',           u'prop_review_score',
                   u'prop_brand_bool',        u'prop_location_score1',
              u'prop_location_score2',   u'prop_log_historical_price',
                         u'price_usd',              u'promotion_flag',
               u'srch_destination_id',         u'srch_length_of_stay',
               u'srch_booking_window',           u'srch_adults_count',
               u'srch_children_count',             u'srch_room_count',
          u'srch_saturday_night_bool',   u'orig_destination_distance',
                       u'random_bool',                      u'target',
                     u'same_location',               u'has_rate_hist',
                   u'comp_rate_lower',              u'comp_inv_worse',
              u'rate_equal_or_better',         u'inv_equal_or_better',
      

### Make sure we have no missing values

In [23]:
missing_values = x.isna().mean() * 100
missing_values

site_id                        0.0
visitor_location_country_id    0.0
prop_country_id                0.0
prop_id                        0.0
prop_starrating                0.0
prop_review_score              0.0
prop_brand_bool                0.0
prop_location_score1           0.0
prop_location_score2           0.0
prop_log_historical_price      0.0
price_usd                      0.0
promotion_flag                 0.0
srch_destination_id            0.0
srch_length_of_stay            0.0
srch_booking_window            0.0
srch_adults_count              0.0
srch_children_count            0.0
srch_room_count                0.0
srch_saturday_night_bool       0.0
orig_destination_distance      0.0
random_bool                    0.0
target                         0.0
same_location                  0.0
has_rate_hist                  0.0
comp_rate_lower                0.0
comp_inv_worse                 0.0
rate_equal_or_better           0.0
inv_equal_or_better            0.0
prop_starrating_mean

### Save the modified parquet up to this point

In [24]:
x.columns=x.columns.astype(str)
x.to_parquet('training_set_v7.parquet.gzip', compression = 'gzip')